In [2]:
# Following example from
# https://www.geeksforgeeks.org/scrape-tables-from-any-website-using-python/

In [41]:
import datetime
import urllib3
import json
import re
from itertools import chain
import time

import matplotlib.pyplot as plt
from pprint import pprint
from html_table_parser.parser import HTMLTableParser
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd


pd.set_option('display.max_columns', None)

---

In [2]:
# First, I want to try and find old auctions for the PHX Auction Co.

In [300]:
def request_auction_details(auction_id):
    """
    Given an auction ID this retrieves the event dates, auctioneer ID and name, and some other miscellaneous data.
    """
    url = 'https://hibid.com/graphql'
    body = json.dumps(
        {
            "operationName": "AuctionDetails",
            "query": "query AuctionDetails($id: Int!, $countAsView: Boolean = true) {\n  auction(id: $id, countAsView: $countAsView) {\n    ...auction\n    __typename\n  }\n}\n\nfragment auction on Auction {\n  id\n  auctioneer {\n    ...auctioneer\n    __typename\n  }\n  eventCity\n  eventDateBegin\n  eventDateEnd\n  eventDateInfo\n  eventName\n  eventState\n  eventZip\n  lotCount\n  showBuyerPremium\n  hidden\n  sourceType\n  __typename\n}\n\nfragment auctioneer on Auctioneer {\n  id\n  name\n  __typename\n}",
            "variables": {
                "countAsView": False,
                "id": auction_id}
        }
    )
    headers = {
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip, deflate, br"
    }
    req = urllib3.PoolManager()
    response = req.request('POST', url, body=body)
    # soup = BeautifulSoup(response.data, 'html.parser')
    if response.status == 200:
        response_data = json.loads(response.data)['data']['auction']
        try:
            auctioneer = response_data.pop('auctioneer')
            response_data['auctioneer_id'] = auctioneer['id']
            response_data['auctioneer_name'] = auctioneer['name']
        except Exception as e:
            print(f'Unable to extract auctioneer for {auction_id}. {repr(e)}')
        return response_data
    else:
        print(f'Unable to retrieve info for {auction_id}. Request returned {response.status} - {response.reason}')
        return None
    

def request_lot_search(auction_id, page_length=1000, page_number=1, url='https://hibid.com/graphql'):
    # TODO: keep auction id in response, need for foreign key matching to auction details table.
    body = json.dumps(
        {
            "operationName": "LotSearch",
            # "query": "query LotSearch($auctionId: Int = null, $pageNumber: Int!, $pageLength: Int!, $category: CategoryId = null, $searchText: String = null, $zip: String = null, $miles: Int = null, $shippingOffered: Boolean = false, $countryName: String = null, $status: AuctionLotStatus = null, $sortOrder: EventItemSortOrder = null, $filter: AuctionLotFilter = null, $isArchive: Boolean = false, $dateStart: DateTime, $dateEnd: DateTime, $countAsView: Boolean = true) {\n  lotSearch(\n    input: {auctionId: $auctionId, category: $category, searchText: $searchText, zip: $zip, miles: $miles, shippingOffered: $shippingOffered, countryName: $countryName, status: $status, sortOrder: $sortOrder, filter: $filter, isArchive: $isArchive, dateStart: $dateStart, dateEnd: $dateEnd, countAsView: $countAsView}\n    pageNumber: $pageNumber\n    pageLength: $pageLength\n    sortDirection: DESC\n  ) {\n    pagedResults {\n      pageLength\n      pageNumber\n      totalCount\n      filteredCount\n      results {\n        bidAmount\n        bidList\n        bidQuantity\n        description\n        estimate\n        featuredPicture {\n          description\n          fullSizeLocation\n          height\n          thumbnailLocation\n          width\n          __typename\n        }\n        forceLiveCatalog\n        fr8StarUrl\n        hideLeadWithDescription\n        id\n        itemId\n        lead\n        links {\n          description\n          id\n          type\n          url\n          videoId\n          __typename\n        }\n        linkTypes\n        lotNumber\n        lotState {\n          bidCount\n          biddingExtended\n          bidMax\n          bidMaxTotal\n          buyerBidStatus\n          buyerHighBid\n          buyerHighBidTotal\n          buyNow\n          choiceType\n          highBid\n          highBuyerId\n          isArchived\n          isClosed\n          isHidden\n          isLive\n          isNotYetLive\n          isOnLiveCatalog\n          isPosted\n          isPublicHidden\n          isRegistered\n          isWatching\n          linkedSoftClose\n          mayHaveWonStatus\n          minBid\n          priceRealized\n          priceRealizedMessage\n          priceRealizedPerEach\n          productStatus\n          productUrl\n          quantitySold\n          reserveSatisfied\n          sealed\n          showBidStatus\n          showReserveStatus\n          softCloseMinutes\n          softCloseSeconds\n          status\n          timeLeft\n          timeLeftLead\n          timeLeftSeconds\n          timeLeftTitle\n          timeLeftWithLimboSeconds\n          timeLeftWithLimboSeconds\n          watchNotes\n          __typename\n        }\n        pictureCount\n        quantity\n        ringNumber\n        rv\n        shippingOffered\n        simulcastStatus\n        site {\n          currencyExpressUrl\n          domain\n          fr8StarUrl\n          isDomainRequest\n          isExtraWWWRequest\n          siteType\n          subdomain\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}",
           # returns only ['description', 'id', 'itemId', 'lead', '__typename']
            "query": "query LotSearch($auctionId: Int = null, $pageNumber: Int!, $pageLength: Int!, $category: CategoryId = null, $searchText: String = null, $zip: String = null, $miles: Int = null, $shippingOffered: Boolean = false, $countryName: String = null, $status: AuctionLotStatus = null, $sortOrder: EventItemSortOrder = null, $filter: AuctionLotFilter = null, $isArchive: Boolean = false, $dateStart: DateTime, $dateEnd: DateTime, $countAsView: Boolean = true) {\n  lotSearch(\n    input: {auctionId: $auctionId, category: $category, searchText: $searchText, zip: $zip, miles: $miles, shippingOffered: $shippingOffered, countryName: $countryName, status: $status, sortOrder: $sortOrder, filter: $filter, isArchive: $isArchive, dateStart: $dateStart, dateEnd: $dateEnd, countAsView: $countAsView}\n    pageNumber: $pageNumber\n    pageLength: $pageLength\n    sortDirection: DESC\n  ) {\n    pagedResults {\n      pageLength\n      pageNumber\n      totalCount\n      filteredCount\n      results {\n        description\n        id\n        itemId\n        lead\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n}",
           # returns 
            "variables": {
                "auctionId": auction_id, #436193,
                "category": "",
                "countAsView": True,
                "countryName": "",
                "filter": "ALL",
                "isArchive": True,
                "miles": -1,
                "pageLength": page_length,
                "pageNumber": page_number,
                "searchText": None,
                "shippingOffered": False,
                "sortOrder": "LOT_NUMBER",
                "status": "ALL",
                "zip": ""
            }
        }
    )
    headers = {
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip, deflate, br"
    }
    req = urllib3.PoolManager()
    response = req.request('POST', url, body=body)
    # soup = BeautifulSoup(response.data, 'html.parser')
    if response.status == 200:
        try:
            response_data = json.loads(response.data)['data']['lotSearch']['pagedResults']['results']
            return response_data
        except Exception as e:
            print(f'Unable to extract data for {auction_id}. {repr(e)}')
    else:
        print(f'Unable to retrieve info for {auction_id}. Request returned {response.status} - {response.reason}')
        

def request_lot_details(lot_id, url='https://hibid.com/graphql', seller_premium=0.15, tax_rate=0.083):
    # Working with graphql request; making request for one specific item
    body = json.dumps(
        {
            "operationName": "GetLotDetails",
            "query": "query GetLotDetails($lotId: ID!, $countAsView: Boolean = true) {\n  lot(input: $lotId, countAsView: $countAsView) {\n    accessability\n    lot {\n      ...lotFull\n      __typename\n    }\n    __typename\n  }\n}\n\n fragment lotFull on Lot {\n  ...lotOnly\n  __typename\n}\n\nfragment lotOnly on Lot {\n  description\n  estimate\n  forceLiveCatalog\n  id\n  itemId\n  lead\n  lotNumber\n  lotState {\n    ...lotState\n    __typename\n  }\n  pictureCount\n  pictures {\n    description\n    fullSizeLocation\n    height\n    thumbnailLocation\n    width\n    __typename\n  }\n  quantity\n  ringNumber\n  rv\n  category {\n    baseCategoryId\n    categoryName\n    description\n    fullCategory\n    header\n    id\n    parentCategoryId\n    __typename\n  }\n  shippingOffered\n  saleOrder\n  __typename\n}\n\nfragment lotState on LotState {\n  bidCount\n  biddingExtended\n  bidMax\n  bidMaxTotal\n  buyerBidStatus\n  buyerHighBid\n  buyerHighBidTotal\n  buyNow\n  choiceType\n  highBid\n  highBuyerId\n  isArchived\n  isClosed\n  isHidden\n  isLive\n  isNotYetLive\n  isOnLiveCatalog\n  isPosted\n  isPublicHidden\n  isRegistered\n  isWatching\n  linkedSoftClose\n  mayHaveWonStatus\n  minBid\n  priceRealized\n  priceRealizedMessage\n  priceRealizedPerEach\n  productStatus\n  productUrl\n  quantitySold\n  reserveSatisfied\n  sealed\n  showBidStatus\n  showReserveStatus\n  softCloseMinutes\n  softCloseSeconds\n  status\n  timeLeft\n  timeLeftLead\n  timeLeftSeconds\n  timeLeftTitle\n  timeLeftWithLimboSeconds\n  timeLeftWithLimboSeconds\n  watchNotes\n  __typename\n}",
            "variables": {
                "countAsView": False,
                "lotId": str(lot_id)
            } #"148810065"}
        }
    )
    headers = {
        "Content-Type": "application/json",
        "Accept-Encoding": "gzip, deflate, br"
    }
    req = urllib3.PoolManager()
    response = req.request('POST', url, body=body)
    if response.status == 200:
        # try:
        response_data = json.loads(response.data)['data']['lot']['lot']
        # Extract all the lot pricing details
        lot_state = response_data.pop('lotState')
        for k, v in lot_state.items():
            response_data[k] = v

        # Extract the star rating
        description = response_data['description']
        try:
            star_grade = re.search('(STAR GRADE\s?-\s?){1}\t[1-5]', description).group().split('\t')[-1]
        except AttributeError:
            try:
                star_grade = re.search('[1-5]\s?-?\s?STAR', description).group().split('STAR')[0]
            except AttributeError as e:
                print(f'Unable to extract star grade for {lot_id}. {repr(e)}')
                star_grade = '0'
        response_data['star_grade'] = int(star_grade.replace('-', '').strip(' '))

        # Extract MSRP value from description
        try:
            # msrp = re.search('(MSRP)\t?\s?(\$[0-9]*.[0-9]*)', description).group().split('$')[-1]
            msrp = float(re.search('(\$[0-9]*.[0-9]*)', description).group().split('$')[-1].replace(',', ''))
        except Exception as e:
            print(f'Unable to extract msrp for {lot_id}. {repr(e)}')
            msrp = None
        response_data['msrp'] = msrp

        # Extract item categories
        categories = response_data.pop('category')
        for i, category in enumerate(categories[::-1]):
            response_data[f'category_{i}'] = category['categoryName']
        response_data['totalPrice'] = round(response_data['highBid'] * (1 + seller_premium + tax_rate), 2)
        return response_data
        # except Exception as e:
        #     print(description)
        #     print(f'Unable to extract data for {lot_id}. {repr(e)}')
    else:
        print(f'Unable to retrieve info for {lot_id}. Request returned {response.status} - {response.reason}')

---

In [61]:
# Working with graphql request; making request for all lots in an auction.
# This query returns results with: ['description', 'id', 'itemId', 'lead', '__typename']
url = 'https://hibid.com/graphql'
body = json.dumps(
    {
        "operationName": "LotSearch",
        # "query": "query LotSearch($auctionId: Int = null, $pageNumber: Int!, $pageLength: Int!, $category: CategoryId = null, $searchText: String = null, $zip: String = null, $miles: Int = null, $shippingOffered: Boolean = false, $countryName: String = null, $status: AuctionLotStatus = null, $sortOrder: EventItemSortOrder = null, $filter: AuctionLotFilter = null, $isArchive: Boolean = false, $dateStart: DateTime, $dateEnd: DateTime, $countAsView: Boolean = true) {\n  lotSearch(\n    input: {auctionId: $auctionId, category: $category, searchText: $searchText, zip: $zip, miles: $miles, shippingOffered: $shippingOffered, countryName: $countryName, status: $status, sortOrder: $sortOrder, filter: $filter, isArchive: $isArchive, dateStart: $dateStart, dateEnd: $dateEnd, countAsView: $countAsView}\n    pageNumber: $pageNumber\n    pageLength: $pageLength\n    sortDirection: DESC\n  ) {\n    pagedResults {\n      pageLength\n      pageNumber\n      totalCount\n      filteredCount\n      results {\n        bidAmount\n        bidList\n        bidQuantity\n        description\n        estimate\n        featuredPicture {\n          description\n          fullSizeLocation\n          height\n          thumbnailLocation\n          width\n          __typename\n        }\n        forceLiveCatalog\n        fr8StarUrl\n        hideLeadWithDescription\n        id\n        itemId\n        lead\n        links {\n          description\n          id\n          type\n          url\n          videoId\n          __typename\n        }\n        linkTypes\n        lotNumber\n        lotState {\n          bidCount\n          biddingExtended\n          bidMax\n          bidMaxTotal\n          buyerBidStatus\n          buyerHighBid\n          buyerHighBidTotal\n          buyNow\n          choiceType\n          highBid\n          highBuyerId\n          isArchived\n          isClosed\n          isHidden\n          isLive\n          isNotYetLive\n          isOnLiveCatalog\n          isPosted\n          isPublicHidden\n          isRegistered\n          isWatching\n          linkedSoftClose\n          mayHaveWonStatus\n          minBid\n          priceRealized\n          priceRealizedMessage\n          priceRealizedPerEach\n          productStatus\n          productUrl\n          quantitySold\n          reserveSatisfied\n          sealed\n          showBidStatus\n          showReserveStatus\n          softCloseMinutes\n          softCloseSeconds\n          status\n          timeLeft\n          timeLeftLead\n          timeLeftSeconds\n          timeLeftTitle\n          timeLeftWithLimboSeconds\n          timeLeftWithLimboSeconds\n          watchNotes\n          __typename\n        }\n        pictureCount\n        quantity\n        ringNumber\n        rv\n        shippingOffered\n        simulcastStatus\n        site {\n          currencyExpressUrl\n          domain\n          fr8StarUrl\n          isDomainRequest\n          isExtraWWWRequest\n          siteType\n          subdomain\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}",
       # returns only ['description', 'id', 'itemId', 'lead', '__typename']
        "query": "query LotSearch($auctionId: Int = null, $pageNumber: Int!, $pageLength: Int!, $category: CategoryId = null, $searchText: String = null, $zip: String = null, $miles: Int = null, $shippingOffered: Boolean = false, $countryName: String = null, $status: AuctionLotStatus = null, $sortOrder: EventItemSortOrder = null, $filter: AuctionLotFilter = null, $isArchive: Boolean = false, $dateStart: DateTime, $dateEnd: DateTime, $countAsView: Boolean = true) {\n  lotSearch(\n    input: {auctionId: $auctionId, category: $category, searchText: $searchText, zip: $zip, miles: $miles, shippingOffered: $shippingOffered, countryName: $countryName, status: $status, sortOrder: $sortOrder, filter: $filter, isArchive: $isArchive, dateStart: $dateStart, dateEnd: $dateEnd, countAsView: $countAsView}\n    pageNumber: $pageNumber\n    pageLength: $pageLength\n    sortDirection: DESC\n  ) {\n    pagedResults {\n      pageLength\n      pageNumber\n      totalCount\n      filteredCount\n      results {\n        description\n        id\n        itemId\n        lead\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n}",
       # returns 
        "variables": {
            "auctionId": 436193,
            "category": "",
            "countAsView": True,
            "countryName": "",
            "filter": "ALL",
            "isArchive": False,
            "miles": -1,
            "pageLength": 700,
            "pageNumber": 1,
            "searchText": None,
            "shippingOffered": False,
            "sortOrder": "LOT_NUMBER",
            "status": "ALL",
            "zip": ""
        }
    }
)
headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip, deflate, br"
}
req = urllib3.PoolManager()
response = req.request('POST', url, body=body)
# soup = BeautifulSoup(response.data, 'html.parser')

response.status

200

In [223]:
auction_details = request_lot_search(436193)
auction_details_df = pd.DataFrame(auction_details)
auction_details_df = auction_details_df.drop(index=[0, 1, 2, 3, 4])

In [224]:
auction_details_df

,description,id,itemId,lead,__typename
5,JBL - PartyBox 110 Portable Party Speaker - Bl...,148810064,7949799,JBL - PartyBox 110 Portable Party Speaker,Lot
6,"Solo Stove Yukon 27"" Smokeless Fire Pit, Natur...",148810065,7953075,"Solo Stove Yukon 27"" Smokeless Fire Pit",Lot
7,*****************5 STARS******************\nMS...,148810082,7958019,American Standard Glenwall VorMax Toilet,Lot
8,"Mongoose Scepter Mountain Bike, 24- "" Wheels, ...",148810117,7958718,Mongoose Scepter Mountain Bike,Lot
9,Walker Edison - L-Shaped Modern Glass Corner C...,148810177,7960763,Walker Edison - L-Shaped Glass Corner Desk,Lot
...,...,...,...,...,...
520,Pickle Ball Game Lawn Sports Set - Sun Squad\n...,148810477,8019761,Pickle Ball Game Lawn Sports Set - Sun Squad,Lot
521,"Homitem Slip on Plush Fleece Slippers, Light G...",148810436,8008857,Homitem Slip on Plush Fleece Slippers,Lot
522,12-Piece Set - Casteline Collection\n\nSTAR GR...,148810442,8009408,CRISTEL CASTELINE COOKWARE,Lot
523,X Rocker - Cobra Gaming Desk with RGB Lighting...,148980258,8050065,X Rocker Cobra Gaming Desk with RGB Lighting,Lot


In [42]:
# Working with graphql request; making request for one specific item
url = 'https://hibid.com/graphql'
body = json.dumps(
    {
        "operationName": "GetLotDetails",
        "query": "query GetLotDetails($lotId: ID!, $countAsView: Boolean = true) {\n  lot(input: $lotId, countAsView: $countAsView) {\n    accessability\n    lot {\n      ...lotFull\n      __typename\n    }\n    __typename\n  }\n}\n\n fragment lotFull on Lot {\n  ...lotOnly\n  __typename\n}\n\nfragment lotOnly on Lot {\n  description\n  estimate\n  forceLiveCatalog\n  id\n  itemId\n  lead\n  lotNumber\n  lotState {\n    ...lotState\n    __typename\n  }\n  pictureCount\n  pictures {\n    description\n    fullSizeLocation\n    height\n    thumbnailLocation\n    width\n    __typename\n  }\n  quantity\n  ringNumber\n  rv\n  category {\n    baseCategoryId\n    categoryName\n    description\n    fullCategory\n    header\n    id\n    parentCategoryId\n    __typename\n  }\n  shippingOffered\n  saleOrder\n  __typename\n}\n\nfragment lotState on LotState {\n  bidCount\n  biddingExtended\n  bidMax\n  bidMaxTotal\n  buyerBidStatus\n  buyerHighBid\n  buyerHighBidTotal\n  buyNow\n  choiceType\n  highBid\n  highBuyerId\n  isArchived\n  isClosed\n  isHidden\n  isLive\n  isNotYetLive\n  isOnLiveCatalog\n  isPosted\n  isPublicHidden\n  isRegistered\n  isWatching\n  linkedSoftClose\n  mayHaveWonStatus\n  minBid\n  priceRealized\n  priceRealizedMessage\n  priceRealizedPerEach\n  productStatus\n  productUrl\n  quantitySold\n  reserveSatisfied\n  sealed\n  showBidStatus\n  showReserveStatus\n  softCloseMinutes\n  softCloseSeconds\n  status\n  timeLeft\n  timeLeftLead\n  timeLeftSeconds\n  timeLeftTitle\n  timeLeftWithLimboSeconds\n  timeLeftWithLimboSeconds\n  watchNotes\n  __typename\n}",
        "variables": {
            "countAsView": False,
            "lotId": "148810065"}
    }
)
headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip, deflate, br"
}
req = urllib3.PoolManager()
response = req.request('POST', url, body=body)
# soup = BeautifulSoup(response.data, 'html.parser')

response.status

200

In [301]:
request_lot_details(148810075)

{'description': "***********************5STARS************************\nMSRP$266.00. ITEM IS NEW.\n\n    Toto Elongated Washlet in Cotton\n\n\nThe TOTO WASHLET A2 Elongated Electronic Bidet Toilet Seat delivers ecology-minded luxury with a streamlined design. Unlike traditional toilet paper that can often be rough and ineffective, the WASHLET quickly provides comfortable warm water cleansing at the touch of a button. The WASHLET A2 is fully automated, featuring a soft rear spray, rear cleanse, and front cleanse with the option of an oscillating stream. The control settings are located on a conveniently attached arm control panel. The spray wand is self-cleaning, intuitively cleaning itself before and after each use. The WASHLET features a heated SoftClose seat that offers three temperature settings. Adjustable water temperature and water pressure with three temperature settings and five pressure settings. Using a WASHLET reduces the need for toilet paper, saving trees and water used to

In [271]:
%%time
lot_details = []
for lot_id in auction_details_df.id.unique():
    lot_data = request_lot_details(lot_id)
    lot_details.append(lot_data)

Unable to extract msrp for 148810130. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810098. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810166. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810121. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810122. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810199. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810211. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810219. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810221. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 148810263. AttributeError("'NoneType' object has no attribute 'group'")
Unable to 

In [272]:
items_df = pd.DataFrame(lot_details)

In [273]:
scraping_date = datetime.datetime.today().date()
items_df.to_csv(f'phoenix_auction_co_tables/all_items_{scraping_date}.csv')

---

In [240]:
%%time
auctions = []
for auction_id in range(400000, 434193):
    auction = request_lot_search(auction_id)
    auctions.append(auction)

ProtocolError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [279]:
lot_search_clean = auctions.copy()

In [267]:
for i, auction in enumerate(auctions):
    for lot in auction:
        lot['auction_id'] = i + 400000

In [268]:
len(auctions)

12099

In [270]:
lots_df = pd.DataFrame(list(chain.from_iterable(auctions)))

In [275]:
lots_df.to_csv('lot_list_auction_id_400000-412098.csv')

---

In [17]:
%%time
auctions = []
for auction_id in range(400000, 434193):
    auction = request_auction_details(auction_id)
    auctions.append(auction)

Unable to extract auctioneer for 400077. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 400451. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 400452. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 400979. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 400980. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 400981. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 400982. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 401026. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 401028. AttributeError("'NoneType' object has no attribute 'pop'")
Unable to extract auctioneer for 401114. AttributeError("'NoneType' object has no attribute 'pop'")


In [20]:
auctions_df = pd.DataFrame([auc for auc in auctions if auc])

In [219]:
auctions_df.to_csv('auction_details_id_400000-434192.csv')

In [43]:
# Auctions held by Phoenix Auction Co.
phx_co_auction_ids = auctions_df.loc[auctions_df.auctioneer_id == 130096].id.unique()
phx_co_auction_ids

array([421698, 422991, 424558, 425910, 427724, 429374, 431051, 432766],
      dtype=uint32)

In [46]:
%%time
phx_co_auctions = []
for auction_id in phx_co_auction_ids:
    print(auction_id)
    phx_co_auction = request_lot_search(int(auction_id))
    for lot in phx_co_auction:
        lot['auction_id'] = auction_id
    phx_co_auctions.append(phx_co_auction)

421698
422991
424558
425910
427724
429374
431051
432766
CPU times: total: 547 ms
Wall time: 5.54 s


In [61]:
phx_auctions_df =  pd.DataFrame(list(chain.from_iterable(phx_co_auctions)))
phx_auctions_df = phx_auctions_df.loc[~phx_auctions_df.lead.isin(
    ['YOU ARE BIDDING IN THE PHX AUCTION CO AUCTION',
     'PHX AUCTION CO TERMS & CONDITIONS',
     'PICKUP DATES AND TIMES',
     'CONDITION FORMAT - PLEASE READ',
     'RETURN POLICY'
    ])
                                     ]
phx_auctions_lot_ids = phx_auctions_df.id.unique()

In [96]:
len(phx_auctions_lot_ids)

3892

In [100]:
%%time
phx_co_lots = []
for lot_id in phx_auctions_lot_ids:
    phx_co_lot = request_lot_details(lot_id)
    phx_co_lots.append(phx_co_lot)

Unable to extract msrp for 142536565. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536260. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536369. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536370. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536355. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536286. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536278. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536608. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536559. AttributeError("'NoneType' object has no attribute 'group'")
Unable to extract msrp for 142536580. AttributeError("'NoneType' object has no attribute 'group'")
Unable to 

In [101]:
len(phx_co_lots)

3892

In [180]:
phx_co_lots_df = pd.DataFrame(phx_co_lots)
phx_co_lots_df = (
    phx_co_lots_df
    .merge(
        phx_auctions_df[['id', 'auction_id']],
        on='id'
    )
)
phx_co_lots_df = (
    phx_co_lots_df
    .merge(
        auctions_df[['id', 'eventDateBegin', 'eventDateEnd']],
        left_on='auction_id',
        right_on='id',
    )
)

In [220]:
phx_co_lots_df.eventDateEnd.value_counts().sort_index()

2023-01-05T00:00:00    433
2023-01-12T00:00:00    464
2023-01-19T00:00:00    422
2023-01-26T00:00:00    496
2023-02-02T00:00:00    474
2023-02-09T00:00:00    523
2023-02-16T00:00:00    462
2023-02-23T00:00:00    618
Name: eventDateEnd, dtype: int64

In [221]:
phx_co_lots_df.to_csv('phoenix_auction_co_tables/all_items_2023-01-05_thru_2023-02-23.csv')

In [182]:
phx_co_lots_df[['category_0', 'category_1']].value_counts(dropna=False).sort_index()

category_0                           category_1                         
Art                                  Paintings                                 1
Business / Retail                    Janitorial                                4
Collectibles                         Decorative                                2
Computers / Consumer Electronics     Consumer Electronics                      1
Farm Equipment                       Motorsports                               1
Furniture                            Cabinets                                  3
                                     Chairs / Stools                           4
                                     Deck / Patio                              7
                                     Desks / Home Office                       2
                                     Dressers / Wardrobes / Armoires           1
                                     Entertainment Centers / TV Stands         2
Machines                            

In [247]:
pc_lead_names = ['EVGA', 'NZXT', 'MSI',  'INTEL', 'NVIDIA', 'GIGABYTE',
                 'GPU', 'CPU', 'POWER SUPPLY', 'MOTHERBOARD'] #, 'MEMORY', 'AMD', 'TOWER'
gaming_lead_names = ['SONY', 'NINTENDO', 'PLAYSTATION', 'XBOX', 'X-BOX', 'SWITCH', 'CONTROLLER', 'GAMING']
tool_lead_names = ['DEWALT', 'RYOBI', 'RIGID', 'MILWAUKEE', 'SNAPON']
vacuum_lead_names = ['SHARK', 'DYSON', 'VACUUM']

pc_regstr = '|'.join(pc_lead_names)
gaming_regstr = '|'.join(gaming_lead_names)
vacuum_regstr = '|'.join(vacuum_lead_names)

In [233]:
phx_co_lots_df[phx_co_lots_df['lead'].str.upper().str.contains(pc_regstr)][['lead', 'star_grade', 'priceRealized', 'msrp', 'eventDateEnd']].sort_values('lead')

,lead,star_grade,priceRealized,msrp,eventDateEnd
2642,"EVGA - 650 BP, 80+ BRONZE 650W, Power supply",5,21.00,74.99,2023-02-09T00:00:00
3863,"EVGA - 650 BP, 80+ BRONZE 650W, 100-BP-0650-K1",5,23.00,74.99,2023-02-23T00:00:00
2779,"EVGA - 650 BP, 80+ BRONZE 650W, 100-BP-0650-K1",5,11.00,74.99,2023-02-09T00:00:00
2371,EVGA - 850W Modular BQ Power Supply - Black,4,56.50,119.99,2023-02-09T00:00:00
2852,EVGA - SuperNOVA 1300W GT Power Supply,5,76.50,269.99,2023-02-16T00:00:00
2727,EVGA - SuperNOVA 1300W GT Power Supply,5,97.50,269.99,2023-02-09T00:00:00
2857,EVGA GT Series 850W Auto Eco Mode with FDB Fan,3,66.00,159.99,2023-02-16T00:00:00
1892,"GIGABYTE - 28"" LED 4K UHD Gaming Monitor (READ)",3,150.00,649.99,2023-02-02T00:00:00
3128,"GIGABYTE - AORUS FV43U 43"" Pro Gaming Monitor",5,296.00,1099.00,2023-02-16T00:00:00
2822,"GIGABYTE - AORUS FV43U 43"" Pro Gaming Monitor",5,234.00,1099.99,2023-02-16T00:00:00


In [291]:
phx_co_lots_df[phx_co_lots_df['lead'].str.upper().str.contains(pc_regstr)]['eventDateEnd'].value_counts(dropna=False).sort_index()

2023-01-12T00:00:00     1
2023-01-26T00:00:00     1
2023-02-02T00:00:00     1
2023-02-09T00:00:00    16
2023-02-16T00:00:00     6
2023-02-23T00:00:00     3
Name: eventDateEnd, dtype: int64

In [202]:
phx_co_lots_df[phx_co_lots_df['lead'].str.upper().str.contains(gaming_regstr) &
              ~phx_co_lots_df['lead'].str.upper().str.contains('SPEAKER|HEADPHONES|CHAIR')][['lead', 'star_grade', 'priceRealized', 'msrp', 'eventDateEnd']].sort_values('lead').head(40)

,lead,star_grade,priceRealized,msrp,eventDateEnd
2429,"ASUS Gaming Monitor -24"" FHD 3D Vision",4,32.5,290.00,2023-02-09T00:00:00
3090,ASUS TUF FreeSync Gaming Monitor,3,49.5,269.99,2023-02-16T00:00:00
2728,"Acer - Predator 27"" IPS Monitor Gaming Monitor",3,69.0,499.00,2023-02-09T00:00:00
3871,"Acer - Predator 27"" IPS Monitor Gaming Monitor",3,91.5,499.00,2023-02-23T00:00:00
2376,Alienware - AW510H Wired 7.1 Gaming Headset,5,16.0,99.00,2023-02-09T00:00:00
2782,Alienware - Stereo Wireless Gaming Headset,4,26.0,199.99,2023-02-09T00:00:00
3702,Alienware - Stereo Wireless Gaming Headset,4,41.5,199.99,2023-02-23T00:00:00
3215,Arozzi - Arena Ultrawide Curved Gaming Desk,4,54.0,499.99,2023-02-16T00:00:00
3838,Astro Gaming - A10 Gen 2 Wired Headset,5,11.0,69.99,2023-02-23T00:00:00
3315,Dell - LED Curved Gaming Monitor,5,235.0,349.99,2023-02-23T00:00:00


In [251]:
phx_co_lots_df[phx_co_lots_df['lead'].str.upper().str.contains('SHARK') &
              phx_co_lots_df['lead'].str.upper().str.contains('VACUUM')][['lead', 'star_grade', 'priceRealized', 'msrp', 'eventDateEnd']].sort_values('lead').head(50)

,lead,star_grade,priceRealized,msrp,eventDateEnd
3551,Shark - AI Ultra 2-in-1 Robot Vacuum and Mop,4,175.00,699.99,2023-02-23T00:00:00
3406,Shark - APEX DuoClean Pet Stick Vacuum,3,16.00,329.00,2023-02-23T00:00:00
1641,Shark - APEX DuoClean Pet Stick Vacuum,3,24.00,329.00,2023-01-26T00:00:00
2961,Shark - Cordless Hand Vacuum,5,71.00,129.99,2023-02-16T00:00:00
3694,Shark - Cordless Pet Plus Upright Vacuum,4,45.00,329.00,2023-02-23T00:00:00
3612,Shark - Cordless Pet Plus Upright Vacuum,4,36.50,329.00,2023-02-23T00:00:00
3417,Shark - Cordless Pet Plus Upright Vacuum,4,29.00,329.00,2023-02-23T00:00:00
2156,Shark - Cordless Pet Plus Vacuum,5,41.00,329.00,2023-02-02T00:00:00
771,Shark - Cordless Pet Plus Vacuum,5,77.50,329.99,2023-01-12T00:00:00
2089,Shark - Cordless Pet Plus Vacuum,5,46.50,329.00,2023-02-02T00:00:00


In [234]:
for i in range(71, 81):
    print(f'{i} group: {i*40} to {(i+1)*40}')
    print(phx_co_lots_df.query('category_0 == "Other"').iloc[i*40:(i+1)*40]['lead'])

71 group: 2840 to 2880
2895           Insignia™ - 20-Cup Rice Cooker and Steamer
2896    Ninja - Mega Kitchen System 72-Oz. Blender - B...
2897                 Insignia™ - 26 Lb. Portable Icemaker
2898    ION Audio - Pathfinder Portable Bluetooth Speaker
2899      Kenwood - Road Series 6-1/2" 2-Way Car Speakers
2900           Shark - Navigator Lift-Away Upright Vacuum
2901            Shark - Cordless Pet Plus Vacuum   - Blue
2902     Dyson - Cyclone V10 Animal Cordless Stick Vacuum
2903        Shark - Vertex UltraLight Corded Stick Vacuum
2904                     "Drifting Tides  Canvas Wall Art
2905             "Ambitious Echo"  Framed Canvas Wall Art
2906          Hampton Bay Matching Base Cabinet End Panel
2907                  LG - CordZero Cordless Stick Vacuum
2908     Walker Edison 59” Urban Industrial L Shaped Desk
2909    Walker Edison Glass Door Cabinet Fireplace TVS...
2910        HARDWOOD REFLECTIONS Birch Wood Butcher Block
2911               Glacier Bay Glamour Corner Sho

In [274]:
current_pc_items = items_df[items_df['lead'].str.upper().str.contains(pc_regstr)]#[['lead', 'star_grade', 'bidCount', 'highBid', 'msrp', 'buyerHighBid', 'buyNow', 'description']].sort_values('lead')

In [277]:
current_pc_items.columns

Index(['description', 'estimate', 'forceLiveCatalog', 'id', 'itemId', 'lead',
       'lotNumber', 'pictureCount', 'pictures', 'quantity', 'ringNumber', 'rv',
       'shippingOffered', 'saleOrder', '__typename', 'bidCount',
       'biddingExtended', 'bidMax', 'bidMaxTotal', 'buyerBidStatus',
       'buyerHighBid', 'buyerHighBidTotal', 'buyNow', 'choiceType', 'highBid',
       'highBuyerId', 'isArchived', 'isClosed', 'isHidden', 'isLive',
       'isNotYetLive', 'isOnLiveCatalog', 'isPosted', 'isPublicHidden',
       'isRegistered', 'isWatching', 'linkedSoftClose', 'mayHaveWonStatus',
       'minBid', 'priceRealized', 'priceRealizedMessage',
       'priceRealizedPerEach', 'productStatus', 'productUrl', 'quantitySold',
       'reserveSatisfied', 'sealed', 'showBidStatus', 'showReserveStatus',
       'softCloseMinutes', 'softCloseSeconds', 'status', 'timeLeft',
       'timeLeftLead', 'timeLeftSeconds', 'timeLeftTitle',
       'timeLeftWithLimboSeconds', 'watchNotes', 'star_grade', 'msrp',
 

In [292]:
current_pc_items['totalPrice'] = current_pc_items['highBid'] * 1.15 * 1.083

C:\Users\shadd\AppData\Local\Temp\ipykernel_22100\3582779877.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_pc_items['totalPrice'] = current_pc_items['highBid'] * 1.15 * 1.083


In [289]:
current_pc_items[['lead', 'star_grade', 'highBid', 'minBid', 'msrp', 'bidCount', 'highBuyerId', 'bidMax', 'saleOrder']].sort_values('lead')

,lead,star_grade,highBid,minBid,msrp,bidCount,highBuyerId,bidMax,saleOrder
446,"EVGA - 650 BP, 80+ BRONZE 650W, Power supply",4,5.0,6.0,74.99,1,3095366,0,496
76,EVGA - GP Series Modular Power Supply,5,9.0,10.0,219.99,2,2521177,0,82
123,"GIGABYTE - AORUS FV43U 43"" Pro Gaming Monitor",5,8.0,9.0,1099.99,4,881585,0,129
337,GIGABYTE - B650M DS3H AMD Motherboard - Black,5,6.0,7.0,159.99,2,6969929,0,369
474,MSI - MPG Z690 FORCE WIFI Socket Intel Motherb...,5,7.0,8.0,369.99,3,3460951,0,544
256,N7 Z690 Intel Motherboard,5,18.0,19.0,299.99,6,1911559,0,268
436,NZXT - AER RGB 2 120mm Fan,4,0.0,5.0,29.99,0,0,0,486
495,NZXT - AER RGB 2 140mm Fan,4,0.0,5.0,32.99,0,0,0,572
426,NZXT - AER RGB 2 140mm Fan,4,0.0,5.0,32.99,0,0,0,476
82,NZXT - C-650 ATX Gaming Power Supply,5,0.0,5.0,129.99,0,0,0,88
